# Final Project: Analyzing U.S. Crime Data

## I) **Data Preparation:**

### I.a) Data Collection:

#### Downloading datasets (First Time only Usage):

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import ttest_ind
from pprint import pprint

In [2]:
from constants import *
from utils import offense_count_df_handling, handle_dataframe, DirectoryManager, FileDownloader

In [3]:
directory_manager = DirectoryManager("data")
file_downloader = FileDownloader(directory_manager)

file_downloader.download_files(urls_wget, urls_gdown)

All files are already downloaded


# Data reading

In [9]:
pers_popdf = pd.read_csv('data/personal_population.csv')
pers_vectimdf = pd.read_csv('data/personal_victimization.csv')

states_popdf = pd.read_csv('data/population_states_1991_2021.csv')
offensecountdf = pd.read_csv('data/offensecountperstate.csv')

fire_lawsdf = pd.read_excel("data/firearm_laws.xlsx")

In [10]:
shape_dict = {
    "personal_population": pers_popdf.shape,
    "personal_victimization": pers_vectimdf.shape,
    "population_states_1991_2021": states_popdf.shape,
    "offense_count_per_state": offensecountdf.shape
}

In [11]:
pprint(shape_dict)

{'offense_count_per_state': (40533, 5),
 'personal_population': (1000000, 20),
 'personal_victimization': (65640, 37),
 'population_states_1991_2021': (51, 32)}


In [12]:
def get_nan_sum(df):
    return df.isna().sum().sum()

nan_sum_dict = {
    "personal_population": get_nan_sum(pers_popdf),
    "personal_victimization": get_nan_sum(pers_vectimdf),
    "population_states_1991_2021": get_nan_sum(states_popdf),
    "offensecountperstate": get_nan_sum(offensecountdf)
}

In [13]:
pprint(nan_sum_dict)

{'offensecountperstate': 0,
 'personal_population': 0,
 'personal_victimization': 1,
 'population_states_1991_2021': 0}


In [14]:
offensecountdf['offenseName'].value_counts().head(10)

theft-from-motor-vehicle                    1766
sex-offenses-non-forcible                   1572
simple-assault                               896
aggravated-assault                           893
all-other-larceny                            893
drug-violations                              893
rape                                         890
burglary-breaking-and-entering               890
destruction-damage-vandalism-of-property     889
fondling                                     889
Name: offenseName, dtype: int64

In [15]:
offensecountdf = offense_count_df_handling(offensecountdf, category_dict, code)
pers_vectimdf = handle_dataframe(pers_vectimdf, "txt_files/pers_vectimdf_column_mappings.json", pers_vectimdf_new_cols_dict)
pers_popdf = handle_dataframe(pers_popdf, "txt_files/pers_popdf_column_mappings.json", pers_popdf_new_cols_dict)

In [16]:
offensecountdf.sample(5)

,stateName,offenseName,offenseCount,Year,offenseCategory,Code
36656,Pennsylvania,weapon-law-violation,40,2020,weapon-law-violation,PA
25831,South Carolina,murder-and-nonnegligent-manslaughter,302,2013,homicide-offenses,SC
19113,Rhode Island,fondling,283,2006,sex-offenses,RI
13027,Mississippi,motor-vehicle-theft,3059,2021,motor-vehicle-theft,MS
29297,Ohio,sex-offenses-non-forcible,53,1999,sex-offenses,OH


# EDA

<h3> Q1: National criminal offense rates per year across all available years for the top five most frequent offense categories.

In [17]:
# Get the top 5 crimes by total count
top_crimes = offensecountdf.groupby("offenseCategory")["offenseCount"].sum().nlargest(5).index

# Pre-compute sums for each year
yearly_totals = offensecountdf.groupby(['Year', 'offenseCategory'])['offenseCount'].sum().unstack()

def get_yearly_percentage(year, crime):
    total_population = states_popdf[str(year)].sum()
    offense_count = yearly_totals.at[year, crime]
    return (offense_count / total_population) * 100_000

# Create traces
fig = go.Figure()

for crime in top_crimes:
    y_data = [get_yearly_percentage(year, crime) for year in range(2021, 1990, -1)]
    fig.add_trace(go.Scatter(x=list(range(2021, 1990, -1)), y=y_data, mode='lines+markers', name=crime))

# Update layout
fig.update_layout(title='Top 5 Crimes by Total Count',
                  xaxis_title='Year',
                  yaxis_title='Offense count per 100k Population',
                  legend=dict(orientation="v", yanchor="bottom", y=1.02, xanchor="right"),
                  margin=dict(l=40, r=40, t=80, b=40),
                  width=1200, height=500,
                  plot_bgcolor='rgba(0,0,0,0)',  # Make plot background transparent
                  xaxis_showgrid=True,  # Show grid for x-axis
                  yaxis_showgrid=True,  # Show grid for y-axis
                  xaxis_gridcolor='LightGrey',  # Color of x-axis gridlines
                  yaxis_gridcolor='LightGrey')  # Color of y-axis gridlines

# Show plot
fig.show()

<h3> Q2: The average percentage of violent crimes relative to total crime per state over all available years.

In [18]:
all_violent_crime = ['assault-offenses', 'homicide-offenses', 'robbery', 'kidnapping-abduction', 'sex-offenses']

# Grouping and summing counts by state and offense category
grouped = offensecountdf.groupby(['Code', "stateName", 'offenseCategory'])['offenseCount'].sum().unstack()
# # Calculating the percentage of violent crimes for each state
grouped['Violent_Crimes_Percentage'] = grouped[all_violent_crime].sum(axis=1) / grouped.sum(axis=1) * 100

# Using Plotly to create a choropleth
grouped = grouped.reset_index()
fig = px.choropleth(
    grouped,  # Resetting index to use columns directly
    locations='Code',  
    color='Violent_Crimes_Percentage',
    hover_name='stateName',  # This will show the full state name on hover
    locationmode="USA-states",  
    scope="usa",
    color_continuous_scale="Reds",
    title="Violent Crimes Percentage by State"
)
fig.add_scattergeo(
    locations=grouped['Code'],
    locationmode='USA-states',
    text=grouped['Code'],
    mode='text')
fig.update_layout(width=1200, height=600, autosize=True)
fig.show()

<h3> Q3: National homicide rates, as well as total violent crime rates per year over all years.

In [19]:
# Grouping and summing counts by year and offense category
grouped_counts = offensecountdf.groupby(['Year', 'offenseCategory'])['offenseCount'].sum().unstack()

# Calculating rates
years = sorted(offensecountdf['Year'].unique())
homicide_rate = {
    year: (grouped_counts.at[year, 'homicide-offenses'] / states_popdf[str(year)].sum()) * 100000
    for year in years
}
total_crimes_rate = {
    year: (grouped_counts.loc[year].sum() / states_popdf[str(year)].sum()) * 100000
    for year in years
}

In [20]:
df = pd.DataFrame({'homicide_rate': pd.Series(homicide_rate), 'total_crimes_rate': pd.Series(total_crimes_rate)})
df.head()

,homicide_rate,total_crimes_rate
1991,0.330064,263.398103
1992,0.288093,337.726691
1993,0.273162,392.510212
1994,0.269453,399.376236
1995,0.196787,373.795256


In [21]:
fig = px.bar(df, y="total_crimes_rate", color="homicide_rate")
fig.update_layout(xaxis_title="Years", title = "Rates per 100k people", width=1200, height=500, autosize=True)
fig.show()

<h3> Q4: The frequency of non-fatal crime incidents in relation to victim demographics.

In [22]:
pers_popdf.columns

Index(['person-id', 'year-and-quarter', 'year', 'age', 'sex',
       'hispanic-origin', 'race', 'race-ethnicity', 'annual-household-income',
       'annual-household-income-imputed', 'marital-status', 'population-size',
       'region', 'household-msa', 'household-locale', 'education-level',
       'education-level-2', 'veteran-status', 'citizenship-status',
       'person-population-weight'],
      dtype='object')

In [23]:
pers_vectimdf.columns

Index(['person-id', 'year-and-quarter', 'year', 'age', 'sex',
       'hispanic-origin', 'race', 'race-ethnicity', 'annual-household-income',
       'annual-household-income-imputed', 'marital-status', 'population-size',
       'region', 'location-of-residence',
       'household-localebjs-geography-definitions', 'education-level',
       'education-level-2', 'veteran-status', 'citizenship-status',
       'aggregate-type-of-crime', 'type-of-crime',
       'violent-crime-excluding-simple-assault', 'reporting-to-police',
       'victim-services', 'location-of-crime', 'victim-offender-relationship',
       'presence-of-weapon', 'weapon-category', 'injury', 'type-of-injury',
       'medical-treatment-for-physical-injuries', 'offender-age',
       'offender-sex', 'offender-race-ethnicity', 'victimization-weight',
       'series', 'series-adjusted-victimization-weight'],
      dtype='object')

In [24]:
# Create subplots with 1 row and 3 columns, specifying the subplot type as 'domain'
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=["Age", "Sex", "Race-ethnicity"])

# Define data and titles
data = [
    (pers_vectimdf["age"], "Age"),
    (pers_vectimdf["sex"], "Sex"),
    (pers_vectimdf["race-ethnicity"], "Race-ethnicity")
]

# Add pie charts
for i, (column, title) in enumerate(data, start=1):
    counts = column.value_counts(normalize=True).sort_index()
    fig.add_trace(go.Pie(labels=counts.index, values=counts.values, name=title), row=1, col=i)

# Update layout
fig.update_layout(title_text="The frequency of non-fatal crime incidents in relation to victim demographics")

# Show plot
fig.show()

<h3> Q5: The frequency of non-fatal crime incidents in relation to offender demographics

In [25]:
# Create subplots with 1 row and 3 columns, specifying the subplot type as 'domain'
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=["Age", "Sex", "Race-ethnicity"])

# Define data and titles
data = [
    (pers_vectimdf["offender-age"], "Age"),
    (pers_vectimdf['offender-sex'], "Sex"),
    (pers_vectimdf['offender-race-ethnicity'], "Race-ethnicity")
]

# Add pie charts
for i, (column, title) in enumerate(data, start=1):
    counts = column.value_counts(normalize=True).sort_index()
    fig.add_trace(go.Pie(labels=counts.index, values=counts.values, name=title), row=1, col=i)

# Update layout
fig.update_layout(title_text="The frequency of non-fatal crime incidents in relation to offender demographics.")

# Show plot
fig.show()

## III) **Answering Questions:**

<h3> Q1:  Which type of non-fatal crime is the most under-reported? Is there an association between the offender-victim relationship and the likelihood of a crime being reported? (reported: ie, police notified at time of occurrence)

In [26]:
# Function to create and customize bar plot
def create_bar_plot(data, title, x_title, y_title):
    fig = px.bar(data, barmode='stack')

    fig.update_layout(
        title=title,
        xaxis_title=x_title,
        yaxis_title=y_title,
        legend_title="Reporting Status",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='white',  # Set plot background color to white
        bargap=0.2,  # Add gap between bars
        font=dict(size=12)  # Set font size for text
    )

    fig.show()

# 1. Which type of non-fatal crime is the most under-reported?
reported_by_crime_type = pers_vectimdf.groupby("type-of-crime")["reporting-to-police"].value_counts(normalize=True).unstack()
create_bar_plot(reported_by_crime_type, "Reporting Status by Crime Type", "Type of Crime", "Proportion")

# 2. Is there an association between the offender-victim relationship and the likelihood of a crime being reported?
reported_by_relationship = pers_vectimdf.groupby('victim-offender-relationship')['reporting-to-police'].value_counts(normalize=True).unstack()
create_bar_plot(reported_by_relationship, "Reporting Status by Offender-Victim Relationship", "Offender-Victim Relationship", "Proportion")

<h3> Q2: Who are the people (the demographic segment) that appear to be most at risk of violent victimization? Who is the least at risk?

In [27]:
# Filter data for violent crime
demographics = pers_vectimdf[pers_vectimdf['aggregate-type-of-crime'] == "Violent crime"]

# Count and normalize age values
age_counts = demographics["age"].value_counts(normalize=True)
# Count and normalize sex values
sex_counts = demographics["sex"].value_counts(normalize=True)
# Count and normalize race-ethnicity values
race_counts = demographics["race-ethnicity"].value_counts(normalize=True)


# Create subplots with 1 row and 3 columns, specifying the subplot type as 'domain'
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=["Age", "Sex", "Race-ethnicity"])

# Add age subplot
fig.add_trace(go.Pie(labels=age_counts.index, values=age_counts.values, name='Age'), row=1, col=1)
# Add sex subplot
fig.add_trace(go.Pie(labels=sex_counts.index, values=sex_counts.values, name='Sex'), row=1, col=2)
# Add race-ethnicity subplot
fig.add_trace(go.Pie(labels=race_counts.index, values=race_counts.values, name='Race-Ethnicity'), row=1, col=3)

# Update layout
fig.update_layout(
    title="Demographics of Victims of Violent Crime",
    grid=dict(rows=1, columns=3),
)

# Show plot
fig.show()

<h3> Q3: Of all victims of non-fatal crimes who suffer an injury, which demographic is the most likely to receive medical attention at the scene? Which is the least likely?

In [28]:
# Filtering data
victims_with_injury = pers_vectimdf[(pers_vectimdf["injury"] == 'Injured') & (pers_vectimdf['medical-treatment-for-physical-injuries'] == "Treated at scene, home, medical office, or other location")]
# Count and normalize age values
age_counts = victims_with_injury["age"].value_counts(normalize=True)
# Count and normalize sex values
sex_counts = victims_with_injury["sex"].value_counts(normalize=True)
# Count and normalize race-ethnicity values
race_counts = victims_with_injury["race-ethnicity"].value_counts(normalize=True)

# Create subplots with 1 row and 3 columns, specifying the subplot type as 'domain'
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=["Age", "Sex", "Race-ethnicity"])

# Add age subplot
fig.add_trace(go.Pie(labels=age_counts.index, values=age_counts.values, name='Age'), row=1, col=1)
# Add sex subplot
fig.add_trace(go.Pie(labels=sex_counts.index, values=sex_counts.values, name='Sex'), row=1, col=2)
# Add race-ethnicity subplot
fig.add_trace(go.Pie(labels=race_counts.index, values=race_counts.values, name='Race-Ethnicity'), row=1, col=3)

# Update layout
fig.update_layout(
    title="Victim Demographics: Non-fatal Crime Injuries Receiving On-scene Medical Attention",
    grid=dict(rows=1, columns=3),
)

# Show plot
fig.show()

<h3> Q4: Which class of crimes is associated with the highest rate of same-offense-recidivism; i.e. prison re-entry for the same offense within 3
years of release?

In [29]:
# Read the data
georgia_df = pd.read_csv('data/georgia_recidivism.csv')

# Filter the data for recidivism within 3 years
crime_recidivism = georgia_df.loc[georgia_df['recidivism_within_3years'] == True]

# Group by prison_offense and count recidivism
crime_group = crime_recidivism.groupby('prison_offense')['recidivism_within_3years'].count().sort_values().reset_index()

# Create Plotly Express bar chart with enhanced features
fig = px.bar(crime_group, 
             x='prison_offense', 
             y='recidivism_within_3years',
             title='Recidivism within 3 years by Prison Offense in Georgia',
             labels={'prison_offense': 'Prison Offense', 'recidivism_within_3years': 'Count'},
             color='prison_offense',  # Color by prison offense
             color_discrete_sequence=px.colors.qualitative.Dark24,  # Use dark colors for better contrast
             template='plotly_white',  # Use white template for clearer background
             hover_name='prison_offense',  # Show prison offense name on hover
             hover_data={'recidivism_within_3years': True},  # Show count on hover
             )

# Update layout for better aesthetics
fig.update_layout(
    xaxis=dict(title='Prison Offense', showgrid=False, tickangle=-45),  # Rotate x-axis labels for better readability
    yaxis=dict(title='Count', showgrid=False),
    font=dict(family="Arial", size=12, color="black"),  # Customize font
    plot_bgcolor='rgba(0,0,0,0)',  # Set plot background color to transparent
    bargap=0.2,  # Set gap between bars
    margin=dict(l=50, r=50, t=70, b=50),  # Adjust margins for better space utilization
)

# Show plot
fig.show()

<h3> Q5: Are prisoners who are younger at the time of release more or less likely to reoffend than those who are older?

In [30]:
# Filter data for recidivism_within_3years == True and False
true_counts = georgia_df.loc[georgia_df["recidivism_within_3years"], "age_at_release"].value_counts(normalize=True).sort_index()
false_counts = georgia_df.loc[~georgia_df["recidivism_within_3years"], "age_at_release"].value_counts(normalize=True).sort_index()

# Create DataFrame for Plotly Express
merged_df = pd.DataFrame({
    'age_at_release': true_counts.index,
    'Reoffend': true_counts.values,
    "Didn't reoffend": false_counts.values
})

fig = px.bar(merged_df, x='age_at_release', y=['Reoffend', "Didn't reoffend"], barmode='group')
fig.update_layout(title="Recidivism within 3 years by Age at Release", xaxis_title="Age at Release", yaxis_title="Normalized Count")
fig.show()

## IV) **Hypothesis Testing:**

In [ ]:
last_10_years = [i for i in range(2010, 2021)]

In [36]:
fire_lawsdf.index = fire_lawsdf.year
fire_laws_last_10_years = fire_lawsdf.loc[last_10_years, ['state', 'lawtotal']].reset_index()
fire_laws_last_10_years.head()

,year,state,lawtotal
0,2010,Alabama,11
1,2010,Alaska,4
2,2010,Arizona,8
3,2010,Arkansas,13
4,2010,California,93


In [32]:
offensecountdf.index = offensecountdf.Year
offense_last_10_years = offensecountdf.loc[last_10_years, ['stateName', 'offenseCount']].reset_index()
offense_last_10_years.rename(columns={'stateName': 'state', "Year": "year"}, inplace=True, errors='raise')
offense_last_10_years.head()

,year,state,offenseCount
0,2010,Delaware,3544
1,2010,Delaware,3664
2,2010,Delaware,301
3,2010,Delaware,16
4,2010,Delaware,9


In [38]:
# Merging the two datasets for last_10_years
merged_last_10_years = pd.merge(fire_laws_last_10_years, offense_last_10_years, on=["year", "state"])

# Compute mean of lawtotal
mean_law_total = merged_last_10_years['lawtotal'].mean()

# Segregate states based on the mean law total
strict_states_last_10_years = merged_last_10_years[merged_last_10_years['lawtotal'] > mean_law_total]
non_strict_states_last_10_years = merged_last_10_years[merged_last_10_years['lawtotal'] <= mean_law_total]

# Performing t-test
t_statistic_last_10_years, p_value_last_10_years = ttest_ind(strict_states_last_10_years['offenseCount'], non_strict_states_last_10_years['offenseCount'])
print("P-value for the last 10 years:", p_value_last_10_years)

P-value for the last 10 years: 0.38295328584572375


# Hypothesis Testing: Relationship Between Firearm Control Laws and Violent Crime Rates in U.S. States

## Hypothesis
The hypothesis being tested is:

"U.S. states that implement stricter firearm control laws have lower violent crime rates on average."

## Analysis Methodology
1. **Data Extraction**: 
   - Relevant data for the last 10 years (2010-2020) was extracted from two datasets:
     - `fire_lawsdf`: Information about firearm control laws
     - `offensecountdf`: Information about offense counts
   - Data was extracted for each year from 2010 to 2020.

2. **Data Preparation**:
   - Data from both datasets were indexed by year.
   - The necessary columns (`state`, `lawtotal` for firearm laws, and `stateName`, `offenseCount` for offense counts) were selected.

3. **Merging Data**:
   - The datasets were merged on both the "year" and "state" columns.

4. **Mean Computation**:
   - The mean total number of firearm control laws across all states over the last 10 years was calculated.

5. **Segregation**:
   - States were divided into two groups based on whether their total number of firearm control laws was greater than or less than or equal to the mean.

6. **Hypothesis Testing**:
   - A t-test was performed to compare offense counts between states with stricter firearm control laws and those with less strict laws over the last 10 years.
   - The p-value obtained from the t-test was used to determine the significance of the relationship between firearm control laws and violent crime rates.

## Results
- **P-value for the Last 10 Years**: 0.383
  - The obtained p-value suggests that there isn't strong statistical evidence to reject the null hypothesis over the last 10 years.
  - Thus, there isn't sufficient evidence to support the claim that U.S. states with stricter firearm control laws have lower violent crime rates on average over this period.